In [2]:
import pandas as pd
import numpy as np
from datetime import datetime
import os
from simulation import Simulation
from glob import glob

In [3]:
data_files = sorted(glob('../../outputs/numpy_arrays/*'))

In [4]:
data_files

['../../outputs/numpy_arrays/simulation_status_quo_2021-11-12_23-32-40-976789.npy']

In [5]:
arr = np.load(data_files[0])

In [6]:
arr.shape

(51, 2, 2, 6)

In [7]:
np.sum(arr, axis=(1,2,3))

array([2.03470319e+08, 1.88430786e+08, 1.88430786e+08, 1.88430786e+08,
       1.88430786e+08, 1.88430786e+08, 1.88430786e+08, 1.88430786e+08,
       1.88430786e+08, 1.88430786e+08, 1.88430786e+08, 1.88430786e+08,
       1.88430786e+08, 1.88430786e+08, 1.88430786e+08, 1.88430786e+08,
       1.88430786e+08, 1.88430786e+08, 1.88430786e+08, 1.88430786e+08,
       1.88430786e+08, 1.88430786e+08, 1.88430786e+08, 1.88430786e+08,
       1.88430786e+08, 1.88430786e+08, 1.88430786e+08, 1.88430786e+08,
       1.88430786e+08, 1.88430786e+08, 1.88430786e+08, 1.88430786e+08,
       1.88430786e+08, 1.88430786e+08, 1.88430786e+08, 1.88430786e+08,
       1.88430786e+08, 1.88430786e+08, 1.88430786e+08, 1.88430786e+08,
       1.88430786e+08, 1.88430786e+08, 1.88430786e+08, 1.88430786e+08,
       1.88430786e+08, 1.88430786e+08, 1.88430786e+08, 1.88430786e+08,
       1.88430786e+08, 1.88430786e+08, 1.88430786e+08])